# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files for English, French, and Spanish movies,
    normalizes their column names, adds a column to denote the original language,
    and concatenates them into a single DataFrame.
    
    Returns:
        pd.DataFrame: A concatenated DataFrame with movies from all specified languages.
    """
     #TODO 1: Add your code here
   
    # Define the paths to the CSV files for each language
    # Ensure the paths are correct. Use absolute paths if the relative paths fail.
    file_paths = {
        "eng": "data/movie_reviews_eng.csv",
        "fr": "data/movie_reviews_fr.csv",
        "sp": "data/movie_reviews_sp.csv",
    }
    
    dfs = []  # To store the DataFrames for each language
    
    for lang, path in file_paths.items():
        try:
            # Attempt to read the CSV file into a DataFrame
            df_tmp = pd.read_csv(path)
        except FileNotFoundError:
            print(f"Error: The file {path} was not found.")
            continue
        except Exception as e:
            print(f"An unexpected error occurred while reading {path}: {e}")
            continue
        
        # Normalize column names to ensure consistency across DataFrames
        # Tip: Check the actual column names in your CSV files and adjust the list accordingly
        df_tmp.columns = ["Title", "Year", "Synopsis", "Review"]
        
        # Add a column to denote the original language of the movie reviews
        df_tmp["Original Language"] = lang
        
        # Append the processed DataFrame to the list
        dfs.append(df_tmp)
    
    if not dfs:
        # If the list is empty, no files were successfully read
        print("Error: No data files were successfully processed.")
        return pd.DataFrame()  # Return an empty DataFrame
    
    # Concatenate all DataFrames into one, resetting the index
    df = pd.concat(dfs, ignore_index=True)

    return df

# Sample a few rows to ensure the data looks correct
df = preprocess_data()
print(df.sample(5))  # Randomly samples 5 rows from the DataFrame

                                 Title  Year  \
18           Les Visiteurs en Amérique  2000   
0            The Shawshank Redemption   1994   
29                        El Incidente  2014   
28  Torrente: El brazo tonto de la ley  1998   
3                       The Godfather   1972   

                                             Synopsis  \
18  Dans cette suite de la comédie française Les V...   
0   Andy Dufresne (Tim Robbins), a successful bank...   
29  En esta película de terror mexicana, un grupo ...   
28  En esta comedia española, un policía corrupto ...   
3   Don Vito Corleone (Marlon Brando) is the head ...   

                                               Review Original Language  
18  "Le film est une perte de temps totale. Les bl...                fr  
0   "The Shawshank Redemption is an inspiring tale...               eng  
29  "El Incidente es una película aburrida y sin s...                sp  
28  "Torrente es una película vulgar y ofensiva qu...                sp 

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",eng
28,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",sp
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",eng
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",fr
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",fr
29,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",sp
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",eng
15,Le Dîner de Cons,1998,Le film suit l'histoire d'un groupe d'amis ric...,"""Je n'ai pas aimé ce film du tout. Le concept ...",fr
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",eng
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",eng


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
from transformers import MarianMTModel, MarianTokenizer

# TODO 2: Update the code below
# Define the model names for French-to-English and Spanish-to-English translation
fr_en_model_name = 'Helsinki-NLP/opus-mt-fr-en'
es_en_model_name = 'Helsinki-NLP/opus-mt-es-en'

# Load the translation models
# Ensure you have an internet connection and access to Hugging Face's model repository
try:
    fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
    es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
except Exception as e:
    print(f"Error loading models: {e}")


# Load the tokenizers for the models
try:
    fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
    es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)
except Exception as e:
    print(f"Error loading tokenizers: {e}")

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    Translates a given text from its original language to English using the specified model and tokenizer.

    Args:
        text (str): The text to translate.
        model: The translation model.
        tokenizer: The tokenizer for the model.

    Returns:
        str: The translated text in English.
    """
    try:
        # Encode the text using the tokenizer
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

        # Generate the translation using the model
        outputs = model.generate(**inputs)

        # Decode the generated output and return the translated text
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return decoded
    except Exception as e:
        print(f"Error during translation: {e}")
        # Return the original text if translation fails, or consider other error handling
        return text

translated_text = translate("Bonjour le monde", fr_en_model, fr_en_tokenizer)
print(translated_text)




/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Hello, the world.


In [5]:
# TODO 4: Update the code below

def translate_reviews_synopsis(df, fr_en_model, fr_en_tokenizer, es_en_model, es_en_tokenizer):
    """
    Filters reviews and synopses in French and Spanish, translates them to English, 
    and updates the DataFrame with the translated text.

    Args:
        df (pd.DataFrame): DataFrame containing the movie reviews and synopses.
        fr_en_model (MarianMTModel): Model for translating French to English.
        fr_en_tokenizer (MarianTokenizer): Tokenizer for the French to English model.
        es_en_model (MarianMTModel): Model for translating Spanish to English.
        es_en_tokenizer (MarianTokenizer): Tokenizer for the Spanish to English model.
    """
    
    # Helper function to translate text based on the original language
    def translate_text_based_on_lang(text, original_lang):
        if original_lang == 'fr':
            return translate(text, fr_en_model, fr_en_tokenizer)
        elif original_lang == 'sp':
            return translate(text, es_en_model, es_en_tokenizer)
        else:
            return text  # Return the original text if it's already in English or undefined language
    
    # Update the DataFrame in place by applying the translation to reviews and synopses
    try:
        df['Review'] = df.apply(lambda x: translate_text_based_on_lang(x['Review'], x['Original Language']), axis=1)
        df['Synopsis'] = df.apply(lambda x: translate_text_based_on_lang(x['Synopsis'], x['Original Language']), axis=1)
        print("Translation completed successfully.")
    except Exception as e:
        print(f"Error during translation process: {e}")
        
    return df


# Apply the translation function to your DataFrame
df = translate_reviews_synopsis(df, fr_en_model, fr_en_tokenizer, es_en_model, es_en_tokenizer)


print(df.sample(5))



/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translation completed successfully.
                             Title  Year  \
6      Scott Pilgrim vs. the World  2010   
7                    The Nice Guys  2016   
23          El Laberinto del Fauno  2006   
16  La Tour Montparnasse Infernale  2001   
13                   Les Choristes  2004   

                                             Synopsis  \
6   Scott Pilgrim (Michael Cera) must defeat his n...   
7   In 1970s Los Angeles, a private eye (Ryan Gosl...   
23  During the Spanish postwar period, Ofelia (Iva...   
16  Two incompetent office workers find themselves...   
13  This film tells the story of a music teacher w...   

                                               Review Original Language  
6   "It was difficult to sit through the whole thi...               eng  
7   "The Nice Guys tries too hard to be funny, and...               eng  
23  "The Labyrinth of Fauno is a fascinating and e...                sp  
16  "I can't believe I've wasted time watching thi...       

In [6]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",eng
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",fr
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",sp
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",eng
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",eng
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",eng
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",sp
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",fr
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",eng


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [7]:
from transformers import pipeline

# TODO 5: Update the code below
# Load sentiment analysis pipeline
# Specify the model name for sentiment analysis
model_name_for_sentiment_analysis = "distilbert-base-uncased-finetuned-sst-2-english"

# Initialize the sentiment analysis pipeline with the specified model
try:
    sentiment_pipeline = pipeline("sentiment-analysis", model=model_name_for_sentiment_analysis)
    print("Sentiment analysis pipeline loaded successfully.")
except Exception as e:
    print(f"Failed to load sentiment analysis pipeline: {e}")

# TODO 6: Complete the function below
def analyze_sentiment(text: str) -> str:
    """
    Analyzes the sentiment of a given text using the sentiment analysis pipeline.

    Args:
        text (str): The text for sentiment analysis.

    Returns:
        str: The sentiment of the text ('POSITIVE' or 'NEGATIVE').
    """
    try:
        result = sentiment_pipeline(text)[0]
        sentiment = result['label']
        return sentiment
    except Exception as e:
        print(f"Error during sentiment analysis: {e}")
        return "UNKNOWN"  # Return UNKNOWN if analysis fails

# Example of analyzing sentiment of a text
example_text = "I love machine learning."
sentiment = analyze_sentiment(example_text)
print(f"Sentiment of the example text: {sentiment}")

# Update the DataFrame with sentiment analysis results
def update_df_with_sentiment(df):
    """
    Updates the DataFrame with sentiment analysis results for each review.

    Args:
        df (pd.DataFrame): The DataFrame containing movie reviews.

    Returns:
        pd.DataFrame: The updated DataFrame with a new column for sentiment results.
    """
    if 'Review' not in df.columns:
        print("The DataFrame does not contain a 'Review' column.")
        return df
    
    df['Sentiment'] = df['Review'].apply(analyze_sentiment)
    return df

# Assuming `df` is your DataFrame
df = update_df_with_sentiment(df)
print(df[['Title', 'Review', 'Sentiment']].head())


Sentiment analysis pipeline loaded successfully.
Sentiment of the example text: POSITIVE
                       Title  \
0  The Shawshank Redemption    
1           The Dark Knight    
2               Forrest Gump   
3             The Godfather    
4                  Inception   

                                              Review Sentiment  
0  "The Shawshank Redemption is an inspiring tale...  POSITIVE  
1  "The Dark Knight is a thrilling and intense su...  POSITIVE  
2  "Forrest Gump is a heartwarming and inspiratio...  POSITIVE  
3  "The Godfather is a classic movie that stands ...  POSITIVE  
4  "Inception is a mind-bending and visually stun...  POSITIVE  


In [8]:
# DONE 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df['Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x)[0])

print(df[['Review', 'Sentiment']].head())


                                              Review Sentiment
0  "The Shawshank Redemption is an inspiring tale...         P
1  "The Dark Knight is a thrilling and intense su...         P
2  "Forrest Gump is a heartwarming and inspiratio...         P
3  "The Godfather is a classic movie that stands ...         P
4  "Inception is a mind-bending and visually stun...         P


In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",eng,N
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",eng,P
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",fr,P
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",sp,P
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp,N
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",eng,P
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",eng,N
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",fr,N
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",eng,N
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",fr,N


In [10]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)